# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [1]:

import tequila as tq
from utility import * #you will need this utility.py from week2
import time

In [2]:
#Running with cell goes over all possible optinons  
molecules=['h2','h4','lih','h2o','n2','nh3']
methods=['hf','cisd','ccsd','fci']
basies=['sto-3g','6-31g']
qubit_transfms=['jw','bk'] 

In [3]:
#Fcus on one molecule
molecules=['lih']
methods=['cisd']
basies=['sto-3g']
qubit_transfms=['jw'] #Jrdan-Wigner transformation is better

In [4]:
R=1.1 #bond_length

for mol in molecules:

    for basis in basies:

        for qubit_transf in qubit_transfms:
                    
            start=time.time()
            
            H = get_qubit_hamiltonian(mol, geometry=R, basis=basis, qubit_transf=qubit_transf)
            print("Take a look at H!")
            
            xyz_data = get_molecular_data(mol, geometry=R, xyz_format=True)
            Htq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

            E_cisd=obtain_PES(mol, [R], basis, 'cisd')
            print('cisd'.upper()," ground state energy is:",E_cisd)
            
            n_spin_orbitals=2*Htq.n_orbitals
            n_electrons=Htq.n_electrons
            print("Numbers of electrons:",n_electrons,
                  "\nUsing ",qubit_transf.upper()," in ",basis.upper(),"basis.",
                  "\nNumber of spin-orbitals (qubits):",n_spin_orbitals,
                  "\nNumber of Hamiltonian terms:",len(H.terms)) 

            H_eff=taper_hamiltonian(H, n_spin_orbitals, n_electrons, qubit_transf)
            print("Number of terms in Taper effective Hamiltonian (H_eff):",len(H_eff.terms))

            
            if mol=='h2':
                # Building the matrix representation of the effective Hamiltonian
                I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
                H_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

                # Obtain the eigenvalues
                eigvals, _ = np.linalg.eigh(H_matrix)
                print("The eigenvalues of the effective Hamiltonian:{}".format(eigvals))
            else:
                print("H_matrix for qubit-tapering technique not yet implemeneted for ",mol.upper())

            print(mol.upper(),"Hamiltonian using ",qubit_transf.upper()," in basis:",
                  basis.upper(),"generated for:",time.time()-start," sec\n")
            
            

Take a look at H!
r=1.1000, E =  -7.82553 Eh
CISD  ground state energy is: [-7.82552855]
Numbers of electrons: 4 
Using  JW  in  STO-3G basis. 
Number of spin-orbitals (qubits): 12 
Number of Hamiltonian terms: 631
Number of terms in Taper effective Hamiltonian (H_eff): 558
H_matrix for qubit-tapering technique not yet implemeneted for  LIH
LIH Hamiltonian using  JW  in basis: STO-3G generated for: 2.4293336868286133  sec



In [5]:
H

-3.9729811446006797 [] +
-0.006570508232166649 [X0 X1 Y2 Y3] +
-0.0033420437049812924 [X0 X1 Y2 Z3 Z4 Y5] +
-0.0009423793397383026 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0033420437049812924 [X0 X1 X3 X4] +
-0.0009423793397383026 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.005047028262495393 [X0 X1 Y4 Y5] +
-0.0016402889577730312 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0016402889577730312 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.0024644863652215263 [X0 X1 Y6 Y7] +
-0.0024644863652215267 [X0 X1 Y8 Y9] +
-0.0009435675808321388 [X0 X1 Y10 Y11] +
0.006570508232166649 [X0 Y1 Y2 X3] +
0.0033420437049812924 [X0 Y1 Y2 Z3 Z4 X5] +
0.0009423793397383026 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0033420437049812924 [X0 Y1 Y3 X4] +
-0.0009423793397383026 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.005047028262495393 [X0 Y1 Y4 X5] +
0.0016402889577730312 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0016402889577730312 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.0024644863652215263 [X0 Y1 Y6 X7] +
0.0024644863652215267 [X0 Y1 Y8 X9]

In [6]:
H_eff

-3.8165083890209752 [] +
0.03770779372081372 [X0] +
-0.002442323717284539 [X0 X1 Y2 Y3] +
0.007488359225978144 [X0 X1 Y2 Z3 Z6 Y7] +
0.001449734245919187 [X0 X1 Z2 X3 Z4 Z5 Z7] +
-0.002868269182600225 [X0 X1 Z2 Z4 Z5 Z6 X7] +
0.007488359225978144 [X0 X1 X3 X6] +
-0.001449734245919187 [X0 X1 X3 Z6 Z7] +
0.006617291077461688 [X0 X1 X4] +
0.006617291077461689 [X0 X1 X5] +
-0.030510577371458618 [X0 X1 Y6 Y7] +
0.002868269182600225 [X0 X1 X7] +
0.001449734245919187 [X0 Y1 Y2] +
0.002442323717284539 [X0 Y1 Y2 X3] +
-0.001449734245919187 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-0.007488359225978144 [X0 Y1 Y2 Z3 Z6 X7] +
0.001449734245919187 [X0 Y1 Z2 Y3 Z4 Z5 Z7] +
-0.002868269182600225 [X0 Y1 Z2 Z3 Y6] +
0.002868269182600225 [X0 Y1 Z2 Z4 Z5 Y6 Z7] +
-0.002868269182600225 [X0 Y1 Z2 Z4 Z5 Z6 Y7] +
0.007488359225978144 [X0 Y1 Y3 X6] +
-0.001449734245919187 [X0 Y1 Y3 Z6 Z7] +
0.006617291077461688 [X0 Y1 Y4] +
0.006617291077461689 [X0 Y1 Y5] +
0.030510577371458618 [X0 Y1 Y6 X7] +
0.002868269182600225 [X0 Y1 Y7]

In [7]:
qwc_list = get_qwc_group(H)
print('Fragments 1: \n{}\n'.format(qwc_list[0]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-3.9729811446006797 [] +
-0.0011013860740815256 [X0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 X8 X9 Z10 X11] +
1.040188976830394 [Z1] +
0.111665814015076 [Z1 Z2] +
0.10509530578290935 [Z1 Z3] +
0.0987929261463902 [Z1 Z4] +
0.0937458978838948 [Z1 Z5] +
0.0990420348338839 [Z1 Z6] +
0.09657754846866236 [Z1 Z7] +
0.09249025929848655 [Z1 Z10] +
-0.09580553083118319 [Z2] +
0.13014781508798245 [Z2 Z3] +
0.05806537505134421 [Z2 Z4] +
0.06050769876862875 [Z2 Z5] +
0.06726816663257461 [Z2 Z6] +
0.0738854577100363 [Z2 Z7] +
0.08469957520253066 [Z2 Z10] +
-0.09580553083118307 [Z3] +
0.06050769876862875 [Z3 Z4] +
0.05806537505134421 [Z3 Z5] +
0.0738854577100363 [Z3 Z6] +
0.06726816663257461 [Z3 Z7] +
0.11521015257398927 [Z3 Z10] +
-0.18979348519160058 [Z4] +
0.0849184164565599 [Z4 Z5] +
0.060179536961667376 [Z4 Z6] +
0.07068226252781408 [Z4 Z7] +
0.05413913932300282 [Z4 Z10] +
-0.18979348519160064 [Z5] +
0.07068226252781408 [Z5 Z6] +
0.060179536961667376 [Z5 Z7] +
0.06082924013706878 [Z5 Z10] +
-0.2193302

In [8]:
comm_groups = get_commuting_group(H)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])
print(comm_groups[2])

Number of mutually commuting fragments: 36
The first commuting group
-3.9729811446006797 [] +
0.006570508232166649 [X0 Y1 Y2 X3] +
-0.00012358305350156356 [X0 Z1 Z2 X3 Y4 Y5] +
-0.002027644410969671 [X0 Z1 Z2 X3 Y6 Y7] +
-0.0020276444109696714 [X0 Z1 Z2 X3 Y8 Y9] +
-0.0015158774207467958 [X0 Z1 Z2 X3 Y10 Y11] +
0.006570508232166649 [Y0 X1 X2 Y3] +
-0.00012358305350156356 [Y0 Z1 Z2 Y3 X4 X5] +
-0.002027644410969671 [Y0 Z1 Z2 Y3 X6 X7] +
-0.0020276444109696714 [Y0 Z1 Z2 Y3 X8 X9] +
-0.0015158774207467958 [Y0 Z1 Z2 Y3 X10 X11] +
0.111665814015076 [Z0 Z3] +
-0.00012358305350156356 [X1 X2 X4 X5] +
-0.002027644410969671 [X1 X2 X6 X7] +
-0.0020276444109696714 [X1 X2 X8 X9] +
-0.0015158774207467956 [X1 X2 X10 X11] +
-0.00012358305350156356 [Y1 Y2 Y4 Y5] +
-0.002027644410969671 [Y1 Y2 Y6 Y7] +
-0.0020276444109696714 [Y1 Y2 Y8 Y9] +
-0.0015158774207467956 [Y1 Y2 Y10 Y11] +
0.111665814015076 [Z1 Z2] +
-0.010502725566146696 [X4 X5 Y6 Y7] +
-0.0105027255661467 [X4 X5 Y8 Y9] +
-0.006690100814065953 

In [9]:
def imp_form4(allz):
    new_form={}
    for keys in allz.terms:
        eeee=list('eeee')
        for key in keys:
            eeee[key[0]]="z"
        new_form["".join(eeee)]=allz.terms[keys]
    return new_form 

In [18]:
def imp_form(allz,n_qubits): #n_spin_orbitals
    new_form={}
    for keys in allz.terms:
        eeee=['e' for i in range(n_qubits)]
        for key in keys:
            eeee[key[0]]="z"
        new_form["".join(eeee)]=allz.terms[keys]
    return new_form 

In [ ]:
%%time
#One has to loop over all groups

new_form=[]
for i in range(1,len(comm_groups)+1):
    uqwc = get_qwc_unitary(comm_groups[i])
    qwc = remove_complex(uqwc * comm_groups[i] * uqwc)

    uz = get_zform_unitary(qwc)
    print("Checking whether U * U^+ is identity: {}".format(uz * uz))

    allz = remove_complex(uz * qwc * uz)
    print("\nThe all-z form of qwc fragment:\n{}".format(allz))
    new_form.append(imp_form(allz,n_spin_orbitals))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-3.9729811446009213 [] +
0.11166581401506738 [Z0] +
0.00012358305350157128 [Z0 Z1 Z2 Z3 Z5] +
0.0020276444109697243 [Z0 Z1 Z2 Z3 Z7] +
0.0020276444109697243 [Z0 Z1 Z2 Z3 Z9] +
0.0015158774207468721 [Z0 Z1 Z2 Z3 Z11] +
-0.00657050823216643 [Z0 Z3] +
0.11166581401506738 [Z1 Z2] +
-0.00657050823216643 [Z1 Z2 Z3] +
0.00012358305350157128 [Z1 Z2 Z3 Z4 Z5] +
0.0020276444109697243 [Z1 Z2 Z3 Z6 Z7] +
0.0020276444109697243 [Z1 Z2 Z3 Z8 Z9] +
0.0015158774207468721 [Z1 Z2 Z3 Z10 Z11] +
-0.00012358305350157128 [Z1 Z2 Z4 Z5] +
-0.0020276444109697243 [Z1 Z2 Z6 Z7] +
-0.0020276444109697243 [Z1 Z2 Z8 Z9] +
-0.0015158774207468721 [Z1 Z2 Z10 Z11] +
0.08491841645655994 [Z4] +
0.010502725566147621 [Z4 Z5 Z7] +
0.010502725566147621 [Z4 Z5 Z9] +
0.0066901008140656006 [Z4 Z5 Z11] +
-0.00012358305350157128 [Z5] +
0.010502725566147621 [Z5 Z6 Z7] +
0.010502725566147621 [Z5 Z8 Z9] +
0.0066901008140656006 [Z5 Z10 Z11] +
0

In [17]:
new_form

[]

In [12]:
n_lements=0
all_togeter={}
for group in new_form:
    print(4,len(group),'real')
    for elemet in group:
        print(elemet,group[elemet])
        if elemet in all_togeter:
            temp=all_togeter[elemet]
            temp+=group[elemet]
            all_togeter[elemet]=temp
        else:
            n_lements+=1
            temp=group[elemet]
            all_togeter[elemet]=temp

print()        
print(4,n_lements,'real',"R=",R,"E[Eh]=",E_cisd[0]," Use the lines below only!")
for key in all_togeter:
    print(key,all_togeter[key])


4 0 real R= 1.1 E[Eh]= -7.825528548150654  Use the lines below only!


In [13]:
-0.3796856633927857+0.15183385960079301

-0.22785180379199269

In [16]:
allz.__dict__

{'terms': {((10, 'Z'),): 0.11140000839963766,
  ((5, 'Z'),): -0.00012358305350157128,
  ((1, 'Z'), (2, 'Z'), (10, 'Z'), (11, 'Z')): -0.0015158774207468721,
  ((11, 'Z'),): -0.0015158774207468721,
  ((1, 'Z'), (2, 'Z'), (4, 'Z'), (5, 'Z')): -0.00012358305350157128,
  ((1, 'Z'), (2, 'Z')): 0.11166581401506738,
  ((0, 'Z'), (3, 'Z')): -0.00657050823216643,
  ((1, 'Z'), (2, 'Z'), (3, 'Z')): -0.00657050823216643,
  ((0, 'Z'),): 0.11166581401506738,
  ((0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z'), (11, 'Z')): 0.0015158774207468721,
  ((1, 'Z'), (2, 'Z'), (3, 'Z'), (10, 'Z'), (11, 'Z')): 0.0015158774207468721,
  ((0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z'), (5, 'Z')): 0.00012358305350157128,
  ((1, 'Z'), (2, 'Z'), (3, 'Z'), (4, 'Z'), (5, 'Z')): 0.00012358305350157128,
  ((4, 'Z'), (5, 'Z'), (11, 'Z')): 0.0066901008140656006,
  ((5, 'Z'), (10, 'Z'), (11, 'Z')): 0.0066901008140656006,
  ((4, 'Z'),): 0.08491841645655994,
  ((7, 'Z'),): -0.0020276444109697243,
  ((1, 'Z'), (2, 'Z'), (6, 'Z'), (7, 'Z')): -0.

In [19]:
imp_form(allz,n_spin_orbitals)

{'eeeeeeeeeeze': 0.11140000839963766,
 'eeeeezeeeeee': -0.00012358305350157128,
 'ezzeeeeeeezz': -0.0015158774207468721,
 'eeeeeeeeeeez': -0.0015158774207468721,
 'ezzezzeeeeee': -0.00012358305350157128,
 'ezzeeeeeeeee': 0.11166581401506738,
 'zeezeeeeeeee': -0.00657050823216643,
 'ezzzeeeeeeee': -0.00657050823216643,
 'zeeeeeeeeeee': 0.11166581401506738,
 'zzzzeeeeeeez': 0.0015158774207468721,
 'ezzzeeeeeezz': 0.0015158774207468721,
 'zzzzezeeeeee': 0.00012358305350157128,
 'ezzzzzeeeeee': 0.00012358305350157128,
 'eeeezzeeeeez': 0.0066901008140656006,
 'eeeeezeeeezz': 0.0066901008140656006,
 'eeeezeeeeeee': 0.08491841645655994,
 'eeeeeeezeeee': -0.0020276444109697243,
 'ezzeeezzeeee': -0.0020276444109697243,
 'ezzeeeeezzee': -0.0020276444109697243,
 'eeeeeeeeezee': -0.0020276444109697243,
 'ezzzeeeezzee': 0.0020276444109697243,
 'eeeezzezeeee': 0.010502725566147621,
 'eeeeezzzeeee': 0.010502725566147621,
 'eeeezzeeezee': 0.010502725566147621,
 'eeeeezeezzee': 0.010502725566147621,
 '

In [ ]:
#This is E0 at R=1 for h2 using H_eff
-0.531051349433764-0.5350572998841723